In [2]:
from manim import *
from numpy import linalg as npl
import math
import matplotlib.pyplot as plt
import jupyter_capture_output

video_scene = " -v WARNING --disable_caching spring_Scene"
image_scene = f" -v WARNING --progress_bar None --disable_caching -r {2*427},{2*240}  -s spring_Scene"

Manim Community v0.17.3

Jupyter Capture Output v0.0.8


In [3]:
# anal2
x0 = -1.5
omega0 = PI / 2

gamma_underdamped = omega0 / 8
gamma_critically_damped = omega0
gamma_overdamped = 4 * omega0


# undampeded oscillation
def y_undamped(t):
    return x0 * np.cos(omega0 * t)


# underdamped oscillation
def y_underdamped(t):
    return x0 * np.cos(np.sqrt(omega0**2-gamma_underdamped) * t) * np.exp(-gamma_underdamped * t)


# critically damped oscillation
def y_critically_damped(t):
    return x0 * (1 + gamma_critically_damped*t) * np.exp(-gamma_critically_damped * t)


# overdamped oscillation 
def y_overdamped(t):
    A = x0 / 2 * (1 + gamma_overdamped / np.sqrt(gamma_overdamped**2 - omega0**2))
    B = x0 / 2 * (1 +- gamma_overdamped / np.sqrt(gamma_overdamped**2 - omega0**2))
    return A * np.exp((-gamma_overdamped + np.sqrt(gamma_overdamped**2 - omega0**2)) * t) + B * np.exp((-gamma_overdamped - np.sqrt(gamma_overdamped**2 - omega0**2)) * t)

In [4]:
# spring model
def Spring(start = UP, end = DOWN, tip_buff = 0.25, nodes = 20, k = 0.5, color = WHITE, stroke_width = 4):
    spring_group = VGroup()
    start[2], end[2] = 0, 0

    # extended parameters
    direction = (end - start) / npl.norm(end - start)
    anti_direction = np.array([direction[1], -direction[0], direction[2]])
    eff_start = start + tip_buff*direction
    eff_end = end - tip_buff*direction
    eff_length = npl.norm(eff_end - eff_start)
    node_length = eff_length / nodes
    node_deviance = k

    # add line with given start 'l_start' and endpoint 'l_end'
    def spring_add_line(l_start, l_end):
        spring_group.add(Line(l_start, l_end, color = color, stroke_width = stroke_width, stroke_opacity = 0.5))
    
    # adding point with given 'pos'
    def spring_add_dot(d_pos):
        spring_group.add(Dot(d_pos, color = color, radius = 0.02))

    # objects
    spring_add_line(start, eff_start)               # start linie
    spring_add_line(eff_end, end)                   # end line
    spring_add_dot(eff_start)                       # start dot
    spring_add_dot(eff_end)                         # end dot
    spring_add_line(eff_start, eff_start + direction * node_length / 2 + anti_direction * node_deviance / 2)                                # first line
    spring_add_dot(eff_start + direction * node_length / 2 + anti_direction * node_deviance / 2)                                            # first dot
    spring_add_line(eff_start + direction * (nodes - 1/2) * node_length + (-1)**(nodes+1) * anti_direction * node_deviance / 2, eff_end)    # last line
    for i in range(1, nodes):
        spring_add_line(eff_start + direction * (i - 1/2) * node_length + (-1)**(i+1) * anti_direction * node_deviance / 2, eff_start + direction * (i + 1/2) * node_length + (-1)**(i) * anti_direction * node_deviance / 2)
        spring_add_dot(eff_start + direction * (i + 1/2) * node_length + (-1)**(i) * anti_direction * node_deviance / 2)
    return spring_group

In [5]:
%%capture_video --path "animations/spring/harmonic_oscillator_ENG_F.mp4"
%%manim -qh --fps 60 $video_scene


class spring_Scene(Scene):
    def construct(self):
        CVC = Text('CVC', font_size = 12, weight = BOLD, color = WHITE, font = 'Latin Modern Sans').align_on_border(RIGHT + DOWN, buff = 0.2)
        self.add(CVC)

        # Header
        text_head = Tex(r"Differential Equations:", font_size = 48).align_on_border(UP + LEFT, buff = 0.5).shift(0.5 * RIGHT + 0.25 * DOWN)
        text_head_ho = Tex(r"Harmonic Oscillator: undamped and damped oscillations", font_size = 48).align_on_border(UP + LEFT, buff = 0.5).shift(0.5 * RIGHT + 0.25 * DOWN)
        text_head_eq1 = Tex(r"$F=-kx$", font_size = 48).next_to(text_head, RIGHT)
        text_head_newton = Tex(r"$=m\ddot{x}$", font_size = 48).next_to(text_head_eq1, RIGHT)
        text_head_eq2 = Tex(r"$F=-kx-b\dot{x}=m\ddot{x}$", font_size = 48).align_on_border(UP + LEFT, buff = 0.5).shift(0.5 * RIGHT + 0.25 * DOWN)

        text_head_x = Tex(r"$\Rightarrow x=-\frac{F}{k}$", font_size = 48).next_to(text_head_newton, 4*RIGHT)
        text_head_x[0][1:2].set_color(RED)
        text_head_x[0][4:7].set_color(BLUE)
        text_head_x_box = SurroundingRectangle(text_head_x[0][1:2], buff = .1, color = RED)
        text_head_Fk_box = SurroundingRectangle(text_head_x[0][4:7], buff = .1, color = BLUE)
        text_head_deq1 = Tex(r"$0=\ddot{x}+\omega_0^2 x,\qquad\omega_0^2:=\frac{k}{m}$", font_size = 48).align_on_border(UP + LEFT, buff = 0.5).shift(0.5 * RIGHT + 0.25 * DOWN)
        text_head_deq2 = Tex(r"$0=\ddot{x}+2\gamma\dot{x}+\omega_0^2 x,\qquad\omega_0^2:=\frac{k}{m},\quad 2\gamma:=\frac{\mu}{m}$", font_size = 48).align_on_border(UP + LEFT, buff = 0.5).shift(0.5 * RIGHT + 0.25 * DOWN)
        
        text_case1 = Tex(r'(a) no damping:', font_size = 32).align_on_border(LEFT, buff = 5).shift(2*UP)
        text_case2 = Tex(r'(b) underdamping ($\gamma^2<\omega_0^2$):', font_size = 32).align_on_border(LEFT, buff = 5).shift(2*UP)
        text_case3 = Tex(r'(c) critical damping ($\gamma^2=\omega_0^2$):', font_size = 32).align_on_border(LEFT, buff = 5).shift(2*UP)
        text_case4 = Tex(r'(d) overdamping ($\gamma^2>\omega_0^2$):', font_size = 32).align_on_border(LEFT, buff = 5).shift(2*UP)


        # Spring
        anchor_pos = np.array([-4.5, 2, 0])
        dot_pos = np.array([-4.5, -1, 0])

        spring = Spring(start = anchor_pos, end = dot_pos, k = 0.5, tip_buff = 0.5, nodes = 20)
        spring_anchor = Line(anchor_pos - 0.5*RIGHT, anchor_pos + 0.5*RIGHT, color = WHITE, stroke_width = 5)
        spring_mass = Dot(dot_pos, color = RED, radius = 0.1)
        spring_base_length = DashedLine(dot_pos - 1*RIGHT, dot_pos + 1*RIGHT, color = WHITE, stroke_width = 4, stroke_opacity = 0.5)

        x_arrow = Line(dot_pos - 0.5*RIGHT, dot_pos - 0.5*RIGHT + x0 *UP, color = RED, stroke_width = 8).add_tip()
        Fk_arrow = Line(dot_pos + 0.5*RIGHT + x0 *UP, dot_pos + 0.5*RIGHT, color = BLUE, stroke_width = 8).add_tip()
        x_brace = Brace(x_arrow, color = RED, direction = np.array([-1, 0, 0]), buff = 0.1, sharpness = 2)
        Fk_brace = Brace(Fk_arrow, color = BLUE, direction = np.array([1, 0, 0]), buff = 0.1, sharpness = 2)
        x_brace_text = Tex(r"$x$", font_size = 48, color = RED).next_to(x_arrow, 0.5*LEFT)
        Fk_brace_text = Tex(r"$\frac{F}{k}$", font_size = 48, color = BLUE).next_to(Fk_arrow, 0.5*RIGHT)
        

        # spring medium
        def medium(density):
            medium_group = VGroup()
            n_strokes = 60
            for i in range(n_strokes):
                medium_group.add(DashedLine(anchor_pos - 1*RIGHT - (4 / (n_strokes-1) * i + 1) * UP, anchor_pos + 1*RIGHT - (4 / (n_strokes-1) * i + 1) * UP, 
                    color = GREY, stroke_opacity = density / 2, stroke_width = 2, dash_length = 0.2, dashed_ratio = 0.95))
            return medium_group

        medium_underdamped = medium(gamma_underdamped * 4)
        medium_critically_damped = medium(gamma_critically_damped)
        medium_overdamped = medium(gamma_overdamped)


        # Coordinate System
        origin = [2, -1, 0]

        x_range = [0, 12, 1]
        y_range = [-2.25, 2.25, 1]

        x_length = 7
        y_length = 4.5

        npla = ComplexPlane(
            x_range = x_range, y_range = y_range, x_length = x_length, y_length = y_length, 
            axis_config = {"include_numbers": False, "stroke_opacity": 0.25}, background_line_style = {"stroke_opacity": 0.25}, ).move_to(origin)#.add_coordinates()
        npla_xlabel = npla.get_x_axis_label(Tex(r"$t$", font_size = 32)).shift(0.4*DOWN)
        npla_ylabel = npla.get_y_axis_label(Tex(r"$x$", font_size = 32))
        npla_group = VGroup(npla, npla_xlabel, npla_ylabel)

        npla2 = ComplexPlane(
            x_range = x_range, y_range = y_range, x_length = 11, y_length = 5, 
            axis_config = {"include_numbers": False, "stroke_opacity": 0.25}, background_line_style = {"stroke_opacity": 0.25}, ).shift(0.5*DOWN)#.add_coordinates()
        npla2_xlabel = npla2.get_x_axis_label(Tex(r"$t$", font_size = 32)).shift(0.4*DOWN)
        npla2_ylabel = npla2.get_y_axis_label(Tex(r"$x$", font_size = 32))
        npla2_group = VGroup(npla2, npla2_xlabel, npla2_ylabel)


        # analytical solutions
        sol_undamped = npla.plot(y_undamped, color = RED, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, 0])
        sol_underdamped = npla.plot(y_underdamped, color = RED, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, 0])
        over_underdamped = npla.plot(lambda x: x0 * np.exp(-gamma_underdamped * x), color = RED, stroke_width = 1.5, stroke_opacity = 0.35, x_range = [0, 0])
        under_underdamped = npla.plot(lambda x: -x0 * np.exp(-gamma_underdamped * x), color = RED, stroke_width = 1.5, stroke_opacity = 0.35, x_range = [0, 0])
        sol_critically_damped = npla.plot(y_critically_damped, color = RED, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, 0])
        sol_overdamped = npla.plot(y_overdamped, color = RED, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, 0])


        # spring updater
        def spring_direct_updater(spring):
            t = y_tracker.get_value()
            target_pos = np.array([dot_pos[0], t, 0])
            spring_mass.become(Dot(target_pos, color = RED, radius = 0.1))
            spring.become(Spring(start = anchor_pos, end = target_pos, k = 0.5, tip_buff = 0.5, nodes = 20))

        # undamped updater
        def spring_undamped_updater(spring):
            t = y_tracker.get_value()
            target_pos = np.array([dot_pos[0], y_undamped(t) - 1, 0])
            sol_undamped.become(npla.plot(y_undamped, color = RED, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, t]))
            spring_mass.become(Dot(target_pos, color = RED, radius = 0.1))
            spring.become(Spring(start = anchor_pos, end = target_pos, k = 0.5, tip_buff = 0.5, nodes = 20))

        # underdamped updater
        def spring_underdamped_updater(spring):
            t = y_tracker.get_value()
            target_pos = np.array([dot_pos[0], y_underdamped(t) - 1, 0])
            sol_underdamped.become(npla.plot(y_underdamped, color = RED, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, t]))
            over_underdamped.become(npla.plot(lambda x: x0 * np.exp(-gamma_underdamped * x), color = RED, stroke_width = 1.5, stroke_opacity = 0.5, x_range = [0, t]))
            under_underdamped.become(npla.plot(lambda x: -x0 * np.exp(-gamma_underdamped * x), color = RED, stroke_width = 1.5, stroke_opacity = 0.5, x_range = [0, t]))
            spring_mass.become(Dot(target_pos, color = RED, radius = 0.1))
            spring.become(Spring(start = anchor_pos, end = target_pos, k = 0.5, tip_buff = 0.5, nodes = 20))

        # critically damped updater
        def spring_critically_damped_updater(spring):
            t = y_tracker.get_value()
            target_pos = np.array([dot_pos[0], y_critically_damped(t) - 1, 0])
            sol_critically_damped.become(npla.plot(y_critically_damped, color = RED, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, t]))
            spring_mass.become(Dot(target_pos, color = RED, radius = 0.1))
            spring.become(Spring(start = anchor_pos, end = target_pos, k = 0.5, tip_buff = 0.5, nodes = 20))

        # overdamped updater
        def spring_overdamped_updater(spring):
            t = y_tracker.get_value()
            target_pos = np.array([dot_pos[0], y_overdamped(t) - 1, 0])
            sol_overdamped.become(npla.plot(y_overdamped, color = RED, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, t]))
            spring_mass.become(Dot(target_pos, color = RED, radius = 0.1))
            spring.become(Spring(start = anchor_pos, end = target_pos, k = 0.5, tip_buff = 0.5, nodes = 20))

        # updates solution in npla2, takes undamped solution
        def sol_updater(sol):
            t = y_tracker.get_value()
            sol_underdamped.become(npla2.plot(y_underdamped, color = BLUE, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, t]))
            sol_critically_damped.become(npla2.plot(y_critically_damped, color = WHITE, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, t]))
            sol_overdamped.become(npla2.plot(y_overdamped, color = RED, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, t]))
            sol.become(npla2.plot(y_undamped, color = GREY, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, t]))


# Fitted Animation PART A
        def PartA():
            self.wait(1.5)
            self.play(Write(text_head), run_time = 2)
            self.wait(1)
            self.play(FadeIn(spring), FadeIn(spring_anchor), FadeIn(spring_base_length), FadeIn(spring_mass), run_time = 3.5)
            self.wait(0.5)
            self.play(FadeOut(spring), run_time = 1.5)
            self.play(FadeIn(spring), run_time = 1.5)
            self.wait(0.5)
            self.play(FadeOut(spring_mass), run_time = 1.5)
            self.play(FadeIn(spring_mass), run_time = 1.5)
            self.wait(1)
            y_tracker.set_value(-1)
            spring.add_updater(spring_direct_updater)
            self.play(y_tracker.animate.set_value(x0-1), FadeIn(x_arrow), Write(x_brace_text), rate_func = smooth, run_time = 2)
            self.wait(5)
            self.play(Write(text_head_eq1), run_time = 1.5)
            self.wait(5)
            self.play(Write(text_head_newton), run_time = 1.5)
            self.wait(13)


# PART B
        def PartB():
            self.add(text_head, spring, spring_anchor, spring_base_length, spring_mass, text_head_eq1, text_head_newton, x_arrow, x_brace_text)
            spring.become(Spring(start = anchor_pos, end = np.array([dot_pos[0], x0-1, 0]), k = 0.5, tip_buff = 0.5, nodes = 20))
            spring_mass.become(Dot(np.array([dot_pos[0], x0-1, 0]), color = RED, radius = 0.1))
            self.wait(2)
            self.play(Write(text_head_x), run_time = 1.5)
            self.wait(3)
            self.play(Create(text_head_Fk_box), run_time = 1.5)
            self.play(TransformFromCopy(text_head_Fk_box, Fk_brace_text), FadeIn(Fk_arrow), run_time = 1.5)
            self.wait(0.5)
            self.play(Uncreate(text_head_Fk_box))
            self.wait(5.5)
            self.play(Transform(VGroup(text_head, text_head_eq1, text_head_newton), text_head_deq1), Unwrite(text_head_x), run_time = 1.5)
            self.wait(1.5)
            self.play(FadeOut(VGroup(x_arrow, Fk_arrow, x_brace_text, Fk_brace_text)), FadeIn(npla_group), run_time = 4.5)
            self.wait(4.5)
            self.play(Write(text_case1), run_time = 1.5)
            self.wait(1.5)
            y_tracker.set_value(0)
            spring.add_updater(spring_undamped_updater)
            self.add(sol_undamped)
            self.play(y_tracker.animate.set_value(12), rate_func = linear, run_time = 6)
            self.wait(3)


# PART C
        def PartC():
            self.add(spring, spring_anchor, spring_base_length, spring_mass, text_head_deq1, npla_group, sol_undamped)
            sol_undamped.become(npla.plot(y_undamped, color = RED, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, 12]))
            spring.become(Spring(start = anchor_pos, end = np.array([dot_pos[0], x0-1, 0]), k = 0.5, tip_buff = 0.5, nodes = 20))
            spring_mass.become(Dot(np.array([dot_pos[0], x0-1, 0]), color = RED, radius = 0.1))
            self.play(Unwrite(text_case1), Unwrite(text_head_deq1), run_time = 1.5)
            self.play(FadeOut(sol_undamped), FadeOut(npla_group), FadeIn(medium_critically_damped), run_time = 3)
            self.wait(1.5)
            self.play(Write(text_head_eq2), run_time = 1.5)
            self.wait(10)
            self.play(Transform(text_head_eq2, text_head_deq2), run_time = 1.5)
            self.wait(3)
            self.play(Transform(medium_critically_damped, medium_underdamped), FadeIn(npla_group), run_time = 4.5)
            self.wait(0.5)
            self.play(Write(text_case2), run_time = 2)
            self.wait(1.5)
            y_tracker.set_value(0)
            spring.add_updater(spring_underdamped_updater)
            self.add(sol_underdamped, over_underdamped, under_underdamped)
            self.play(y_tracker.animate.set_value(12), rate_func = linear, run_time = 6)
            self.wait(3)


# PART D
        def PartD():
            self.add(spring, spring_anchor, spring_base_length, spring_mass, text_head_deq2, npla_group, medium_underdamped, sol_underdamped, over_underdamped, under_underdamped, text_case2)
            sol_underdamped.become(npla.plot(y_underdamped, color = RED, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, 12]))
            over_underdamped.become(npla.plot(lambda x: x0 * np.exp(-gamma_underdamped * x), color = RED, stroke_width = 1.5, stroke_opacity = 0.5, x_range = [0, 12]))
            under_underdamped.become(npla.plot(lambda x: -x0 * np.exp(-gamma_underdamped * x), color = RED, stroke_width = 1.5, stroke_opacity = 0.5, x_range = [0, 12]))
            y_tracker.set_value(-1+y_underdamped(12))
            spring.add_updater(spring_direct_updater)
            self.play(FadeOut(sol_underdamped), FadeOut(over_underdamped), FadeOut(under_underdamped), Unwrite(text_case2), run_time = 1.5)
            self.play(y_tracker.animate.set_value(x0-1), rate_func = smooth, run_time = 2)
            self.play(Transform(medium_underdamped, medium_critically_damped), run_time = 3)
            self.wait(0.5)
            self.play(Write(text_case3), run_time = 1.5)
            self.wait(1.5)
            self.remove_updater(spring_direct_updater)
            y_tracker.set_value(0)
            spring.add_updater(spring_critically_damped_updater)
            self.add(sol_critically_damped)
            self.play(y_tracker.animate.set_value(12), rate_func = linear, run_time = 6)
            self.wait(3)


# PART E
        def PartE():
            self.add(spring, spring_anchor, spring_base_length, spring_mass, text_head_deq2, npla_group, medium_critically_damped, sol_critically_damped, text_case3)
            sol_critically_damped.become(npla.plot(y_critically_damped, color = RED, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, 12]))
            y_tracker.set_value(-1+y_critically_damped(12))
            spring.add_updater(spring_direct_updater)
            self.play(FadeOut(sol_critically_damped), Unwrite(text_case3), run_time = 1.5)
            self.play(y_tracker.animate.set_value(x0-1), rate_func = smooth, run_time = 2)
            self.play(Transform(medium_critically_damped, medium_overdamped), run_time = 3)
            self.wait(0.5)
            self.play(Write(text_case4), run_time = 1.5)
            self.wait(1.5)
            self.remove_updater(spring_direct_updater)
            y_tracker.set_value(0)
            spring.add_updater(spring_overdamped_updater)
            self.add(sol_overdamped)
            self.play(y_tracker.animate.set_value(12), rate_func = linear, run_time = 6)
            self.wait(3)


# PART F
        def Part(F):
            self.add(spring, spring_anchor, spring_base_length, spring_mass, text_head_deq2, npla_group, medium_overdamped, sol_overdamped, text_case4)
            spring.become(Spring(start = anchor_pos, end = np.array([dot_pos[0], y_overdamped(12) - 1, 0]), k = 0.5, tip_buff = 0.5, nodes = 20))
            spring_mass.become(Dot(np.array([dot_pos[0], y_overdamped(12) - 1, 0]), color = RED, radius = 0.1))
            self.play(Unwrite(text_case4), Unwrite(text_head_deq2), FadeOut(sol_overdamped), FadeOut(spring), FadeOut(spring_anchor), FadeOut(spring_base_length), FadeOut(spring_mass), FadeOut(medium_overdamped), run_time = 1.5)
            self.wait(1)
            self.play(Write(text_head_ho), Transform(npla_group, npla2_group), run_time = 1.5)
            self.wait(5)
            self.add(sol_undamped, sol_underdamped, sol_critically_damped, sol_overdamped)
            sol_undamped.add_updater(sol_updater)
            y_tracker.set_value(0)
            self.play(y_tracker.animate.set_value(12), rate_func = linear, run_time = 6)
            self.wait(10)


        y_tracker = ValueTracker(0)

Output saved by overwring previous file at animations/spring/harmonic_oscillator_F.mp4.


### Audio Description

[0:00] Todays topic is differential equations. [0:04] For that matter an ordinary spring will serve as our oscillating model system. [0:09] The spring is shown on the left side of the frame, a red mass is attached to its tail.

[0:15] We now extend the spring by a certain distance and observe the acting forces.

[0:21] According to Hooke's law we get minus the spring constant times the extension.

[0:27] With Newton's second law - eff equals the mass times the acceleration - we obtain the full equation displayed at the top.

[0:36] Such an equation - including a function like ex as well as one or more of its derivatives - is called differential equation.

---

[0:44] We can now look at what this differential equation means physically for the extension ex.

[0:50] The resulting force is proportional to ex and pointed into the opposite direction.

[0:56] Having this established, we will now write down the differential equation in an alternative form. 

[1:04] Possbile solutions follow directly from letting the spring oscillate out of its current position.

[1:09] The simplest case does not include any damping and would therefore oscillate essentially forever.

---

[1:24] Next, we imagine a medium around the springs path.

[1:28] Our original force receive an additional term for the friction.

[1:33] Friction terms are proportional to the velocity and oppose it in terms of direction.

[1:39] Again, we rewrite our differential equation using appropriate substitutions.

[1:44] We now look at the solution for a thin medium. A weakly damped harmonic oscillator.

[1:53] The amplitudes are exponentially decreasing over time, but an oscillation is still visible. 

---

[2:04] While we move the spring back to its initial extension, the medium is becoming more dense. 

[2:13] The spring is now critically damped, the oscillation vanishes. 

---

The density of the medium can now be increased even further.

Of this overdamped harmonic oscillator, the solution is also shown below.

---

In the end, we will now put all four solutions in one single coordinate system to summarize the harmonic oscillator.

While the solutions are not labeled, you should be able to recognize the different cases easily after this video.

Thanks for watching and have a nice day.

In [15]:
%%capture_video --path "animations/spring/harmonic_oscillator_GER_F.mp4"
%%manim -qh --fps 60 $video_scene


class spring_Scene(Scene):
    def construct(self):
        CVC = Text('CVC', font_size = 12, weight = BOLD, color = WHITE, font = 'Latin Modern Sans').align_on_border(RIGHT + DOWN, buff = 0.2)
        self.add(CVC)

        # Header
        text_head = Tex(r"Differentialgleichungen:", font_size = 48).align_on_border(UP + LEFT, buff = 0.5).shift(0.5 * RIGHT + 0.25 * DOWN)
        text_head_ho = Tex(r"Harmonischer Oszillator: Zusammenfassung", font_size = 48).align_on_border(UP + LEFT, buff = 0.5).shift(0.5 * RIGHT + 0.25 * DOWN)
        text_head_eq1 = Tex(r"$F=-kx$", font_size = 48).next_to(text_head, RIGHT)
        text_head_newton = Tex(r"$=m\ddot{x}$", font_size = 48).next_to(text_head_eq1, RIGHT)
        text_head_eq2 = Tex(r"$F=-kx-b\dot{x}=m\ddot{x}$", font_size = 48).align_on_border(UP + LEFT, buff = 0.5).shift(0.5 * RIGHT + 0.25 * DOWN)

        text_head_x = Tex(r"$\Rightarrow x=-\frac{F}{k}$", font_size = 48).next_to(text_head_newton, 4*RIGHT)
        text_head_x[0][1:2].set_color(RED)
        text_head_x[0][4:7].set_color(BLUE)
        text_head_x_box = SurroundingRectangle(text_head_x[0][1:2], buff = .1, color = RED)
        text_head_Fk_box = SurroundingRectangle(text_head_x[0][4:7], buff = .1, color = BLUE)
        text_head_deq1 = Tex(r"$0=\ddot{x}+\omega_0^2 x,\qquad\omega_0^2:=\frac{k}{m}$", font_size = 48).align_on_border(UP + LEFT, buff = 0.5).shift(0.5 * RIGHT + 0.25 * DOWN)
        text_head_deq2 = Tex(r"$0=\ddot{x}+2\gamma\dot{x}+\omega_0^2 x,\qquad\omega_0^2:=\frac{k}{m},\quad 2\gamma:=\frac{\mu}{m}$", font_size = 48).align_on_border(UP + LEFT, buff = 0.5).shift(0.5 * RIGHT + 0.25 * DOWN)
        
        text_case1 = Tex(r'(a) keine Dämpfung:', font_size = 32).align_on_border(LEFT, buff = 5).shift(2*UP)
        text_case2 = Tex(r'(b) schwache Dämpfung ($\gamma^2<\omega_0^2$):', font_size = 32).align_on_border(LEFT, buff = 5).shift(2*UP)
        text_case3 = Tex(r'(c) aperiodischer Grenzfall ($\gamma^2=\omega_0^2$):', font_size = 32).align_on_border(LEFT, buff = 5).shift(2*UP)
        text_case4 = Tex(r'(d) starke Dämfung ($\gamma^2>\omega_0^2$):', font_size = 32).align_on_border(LEFT, buff = 5).shift(2*UP)


        # Spring
        anchor_pos = np.array([-4.5, 2, 0])
        dot_pos = np.array([-4.5, -1, 0])

        spring = Spring(start = anchor_pos, end = dot_pos, k = 0.5, tip_buff = 0.5, nodes = 20)
        spring_anchor = Line(anchor_pos - 0.5*RIGHT, anchor_pos + 0.5*RIGHT, color = WHITE, stroke_width = 5)
        spring_mass = Dot(dot_pos, color = RED, radius = 0.1)
        spring_base_length = DashedLine(dot_pos - 1*RIGHT, dot_pos + 1*RIGHT, color = WHITE, stroke_width = 4, stroke_opacity = 0.5)

        x_arrow = Line(dot_pos - 0.5*RIGHT, dot_pos - 0.5*RIGHT + x0 *UP, color = RED, stroke_width = 8).add_tip()
        Fk_arrow = Line(dot_pos + 0.5*RIGHT + x0 *UP, dot_pos + 0.5*RIGHT, color = BLUE, stroke_width = 8).add_tip()
        x_brace = Brace(x_arrow, color = RED, direction = np.array([-1, 0, 0]), buff = 0.1, sharpness = 2)
        Fk_brace = Brace(Fk_arrow, color = BLUE, direction = np.array([1, 0, 0]), buff = 0.1, sharpness = 2)
        x_brace_text = Tex(r"$x$", font_size = 48, color = RED).next_to(x_arrow, 0.5*LEFT)
        Fk_brace_text = Tex(r"$\frac{F}{k}$", font_size = 48, color = BLUE).next_to(Fk_arrow, 0.5*RIGHT)
        

        # spring medium
        def medium(density):
            medium_group = VGroup()
            n_strokes = 60
            for i in range(n_strokes):
                medium_group.add(DashedLine(anchor_pos - 1*RIGHT - (4 / (n_strokes-1) * i + 1) * UP, anchor_pos + 1*RIGHT - (4 / (n_strokes-1) * i + 1) * UP, 
                    color = GREY, stroke_opacity = density / 2, stroke_width = 2, dash_length = 0.2, dashed_ratio = 0.95))
            return medium_group

        medium_underdamped = medium(gamma_underdamped * 4)
        medium_critically_damped = medium(gamma_critically_damped)
        medium_overdamped = medium(gamma_overdamped)


        # Coordinate System
        origin = [2, -1, 0]

        x_range = [0, 12, 1]
        y_range = [-2.25, 2.25, 1]

        x_length = 7
        y_length = 4.5

        npla = ComplexPlane(
            x_range = x_range, y_range = y_range, x_length = x_length, y_length = y_length, 
            axis_config = {"include_numbers": False, "stroke_opacity": 0.25}, background_line_style = {"stroke_opacity": 0.25}, ).move_to(origin)#.add_coordinates()
        npla_xlabel = npla.get_x_axis_label(Tex(r"$t$", font_size = 32)).shift(0.4*DOWN)
        npla_ylabel = npla.get_y_axis_label(Tex(r"$x$", font_size = 32))
        npla_group = VGroup(npla, npla_xlabel, npla_ylabel)

        npla2 = ComplexPlane(
            x_range = x_range, y_range = y_range, x_length = 11, y_length = 5, 
            axis_config = {"include_numbers": False, "stroke_opacity": 0.25}, background_line_style = {"stroke_opacity": 0.25}, ).shift(0.5*DOWN)#.add_coordinates()
        npla2_xlabel = npla2.get_x_axis_label(Tex(r"$t$", font_size = 32)).shift(0.4*DOWN)
        npla2_ylabel = npla2.get_y_axis_label(Tex(r"$x$", font_size = 32))
        npla2_group = VGroup(npla2, npla2_xlabel, npla2_ylabel)


        # analytical solutions
        sol_undamped = npla.plot(y_undamped, color = RED, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, 0])
        sol_underdamped = npla.plot(y_underdamped, color = RED, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, 0])
        over_underdamped = npla.plot(lambda x: x0 * np.exp(-gamma_underdamped * x), color = RED, stroke_width = 1.5, stroke_opacity = 0.35, x_range = [0, 0])
        under_underdamped = npla.plot(lambda x: -x0 * np.exp(-gamma_underdamped * x), color = RED, stroke_width = 1.5, stroke_opacity = 0.35, x_range = [0, 0])
        sol_critically_damped = npla.plot(y_critically_damped, color = RED, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, 0])
        sol_overdamped = npla.plot(y_overdamped, color = RED, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, 0])


        # spring updater
        def spring_direct_updater(spring):
            t = y_tracker.get_value()
            target_pos = np.array([dot_pos[0], t, 0])
            spring_mass.become(Dot(target_pos, color = RED, radius = 0.1))
            spring.become(Spring(start = anchor_pos, end = target_pos, k = 0.5, tip_buff = 0.5, nodes = 20))

        # undamped updater
        def spring_undamped_updater(spring):
            t = y_tracker.get_value()
            target_pos = np.array([dot_pos[0], y_undamped(t) - 1, 0])
            sol_undamped.become(npla.plot(y_undamped, color = RED, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, t]))
            spring_mass.become(Dot(target_pos, color = RED, radius = 0.1))
            spring.become(Spring(start = anchor_pos, end = target_pos, k = 0.5, tip_buff = 0.5, nodes = 20))

        # underdamped updater
        def spring_underdamped_updater(spring):
            t = y_tracker.get_value()
            target_pos = np.array([dot_pos[0], y_underdamped(t) - 1, 0])
            sol_underdamped.become(npla.plot(y_underdamped, color = RED, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, t]))
            over_underdamped.become(npla.plot(lambda x: x0 * np.exp(-gamma_underdamped * x), color = RED, stroke_width = 1.5, stroke_opacity = 0.5, x_range = [0, t]))
            under_underdamped.become(npla.plot(lambda x: -x0 * np.exp(-gamma_underdamped * x), color = RED, stroke_width = 1.5, stroke_opacity = 0.5, x_range = [0, t]))
            spring_mass.become(Dot(target_pos, color = RED, radius = 0.1))
            spring.become(Spring(start = anchor_pos, end = target_pos, k = 0.5, tip_buff = 0.5, nodes = 20))

        # critically damped updater
        def spring_critically_damped_updater(spring):
            t = y_tracker.get_value()
            target_pos = np.array([dot_pos[0], y_critically_damped(t) - 1, 0])
            sol_critically_damped.become(npla.plot(y_critically_damped, color = RED, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, t]))
            spring_mass.become(Dot(target_pos, color = RED, radius = 0.1))
            spring.become(Spring(start = anchor_pos, end = target_pos, k = 0.5, tip_buff = 0.5, nodes = 20))

        # overdamped updater
        def spring_overdamped_updater(spring):
            t = y_tracker.get_value()
            target_pos = np.array([dot_pos[0], y_overdamped(t) - 1, 0])
            sol_overdamped.become(npla.plot(y_overdamped, color = RED, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, t]))
            spring_mass.become(Dot(target_pos, color = RED, radius = 0.1))
            spring.become(Spring(start = anchor_pos, end = target_pos, k = 0.5, tip_buff = 0.5, nodes = 20))

        # updates solution in npla2, takes undamped solution
        def sol_updater(sol):
            t = y_tracker.get_value()
            sol_underdamped.become(npla2.plot(y_underdamped, color = BLUE, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, t]))
            sol_critically_damped.become(npla2.plot(y_critically_damped, color = WHITE, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, t]))
            sol_overdamped.become(npla2.plot(y_overdamped, color = RED, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, t]))
            sol.become(npla2.plot(y_undamped, color = GREY, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, t]))


# Fitted Animation PART A
        def PartA():
            self.wait(1.5)
            self.play(Write(text_head), run_time = 2)
            self.wait(1)
            self.play(FadeIn(spring), FadeIn(spring_anchor), FadeIn(spring_base_length), FadeIn(spring_mass), run_time = 3.5)
            self.wait(0.5)
            self.play(FadeOut(spring), run_time = 1.5)
            self.play(FadeIn(spring), run_time = 1.5)
            self.wait(0.5)
            self.play(FadeOut(spring_mass), run_time = 1.5)
            self.play(FadeIn(spring_mass), run_time = 1.5)
            self.wait(1)
            y_tracker.set_value(-1)
            spring.add_updater(spring_direct_updater)
            self.play(y_tracker.animate.set_value(x0-1), FadeIn(x_arrow), Write(x_brace_text), rate_func = smooth, run_time = 2)
            self.wait(5)
            self.play(Write(text_head_eq1), run_time = 1.5)
            self.wait(5)
            self.play(Write(text_head_newton), run_time = 1.5)
            self.wait(13)


# PART B
        def PartB():
            self.add(text_head, spring, spring_anchor, spring_base_length, spring_mass, text_head_eq1, text_head_newton, x_arrow, x_brace_text)
            spring.become(Spring(start = anchor_pos, end = np.array([dot_pos[0], x0-1, 0]), k = 0.5, tip_buff = 0.5, nodes = 20))
            spring_mass.become(Dot(np.array([dot_pos[0], x0-1, 0]), color = RED, radius = 0.1))
            self.wait(2)
            self.play(Write(text_head_x), run_time = 1.5)
            self.wait(3)
            self.play(Create(text_head_Fk_box), run_time = 1.5)
            self.play(TransformFromCopy(text_head_Fk_box, Fk_brace_text), FadeIn(Fk_arrow), run_time = 1.5)
            self.wait(0.5)
            self.play(Uncreate(text_head_Fk_box))
            self.wait(5.5)
            self.play(Transform(VGroup(text_head, text_head_eq1, text_head_newton), text_head_deq1), Unwrite(text_head_x), run_time = 1.5)
            self.wait(1.5)
            self.play(FadeOut(VGroup(x_arrow, Fk_arrow, x_brace_text, Fk_brace_text)), FadeIn(npla_group), run_time = 4.5)
            self.wait(4.5)
            self.play(Write(text_case1), run_time = 1.5)
            self.wait(1.5)
            y_tracker.set_value(0)
            spring.add_updater(spring_undamped_updater)
            self.add(sol_undamped)
            self.play(y_tracker.animate.set_value(12), rate_func = linear, run_time = 6)
            self.wait(3)


# PART C
        def PartC():
            self.add(spring, spring_anchor, spring_base_length, spring_mass, text_head_deq1, npla_group, sol_undamped)
            sol_undamped.become(npla.plot(y_undamped, color = RED, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, 12]))
            spring.become(Spring(start = anchor_pos, end = np.array([dot_pos[0], x0-1, 0]), k = 0.5, tip_buff = 0.5, nodes = 20))
            spring_mass.become(Dot(np.array([dot_pos[0], x0-1, 0]), color = RED, radius = 0.1))
            self.play(Unwrite(text_case1), Unwrite(text_head_deq1), run_time = 1.5)
            self.play(FadeOut(sol_undamped), FadeOut(npla_group), FadeIn(medium_critically_damped), run_time = 3)
            self.wait(1.5)
            self.play(Write(text_head_eq2), run_time = 1.5)
            self.wait(10)
            self.play(Transform(text_head_eq2, text_head_deq2), run_time = 1.5)
            self.wait(3)
            self.play(Transform(medium_critically_damped, medium_underdamped), FadeIn(npla_group), run_time = 4.5)
            self.wait(0.5)
            self.play(Write(text_case2), run_time = 2)
            self.wait(1.5)
            y_tracker.set_value(0)
            spring.add_updater(spring_underdamped_updater)
            self.add(sol_underdamped, over_underdamped, under_underdamped)
            self.play(y_tracker.animate.set_value(12), rate_func = linear, run_time = 6)
            self.wait(3)


# PART D
        def PartD():
            self.add(spring, spring_anchor, spring_base_length, spring_mass, text_head_deq2, npla_group, medium_underdamped, sol_underdamped, over_underdamped, under_underdamped, text_case2)
            sol_underdamped.become(npla.plot(y_underdamped, color = RED, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, 12]))
            over_underdamped.become(npla.plot(lambda x: x0 * np.exp(-gamma_underdamped * x), color = RED, stroke_width = 1.5, stroke_opacity = 0.5, x_range = [0, 12]))
            under_underdamped.become(npla.plot(lambda x: -x0 * np.exp(-gamma_underdamped * x), color = RED, stroke_width = 1.5, stroke_opacity = 0.5, x_range = [0, 12]))
            y_tracker.set_value(-1+y_underdamped(12))
            spring.add_updater(spring_direct_updater)
            self.play(FadeOut(sol_underdamped), FadeOut(over_underdamped), FadeOut(under_underdamped), Unwrite(text_case2), run_time = 1.5)
            self.play(y_tracker.animate.set_value(x0-1), rate_func = smooth, run_time = 2)
            self.play(Transform(medium_underdamped, medium_critically_damped), run_time = 3)
            self.wait(0.5)
            self.play(Write(text_case3), run_time = 1.5)
            self.wait(1.5)
            self.remove_updater(spring_direct_updater)
            y_tracker.set_value(0)
            spring.add_updater(spring_critically_damped_updater)
            self.add(sol_critically_damped)
            self.play(y_tracker.animate.set_value(12), rate_func = linear, run_time = 6)
            self.wait(3)


# PART E
        def PartE():
            self.add(spring, spring_anchor, spring_base_length, spring_mass, text_head_deq2, npla_group, medium_critically_damped, sol_critically_damped, text_case3)
            sol_critically_damped.become(npla.plot(y_critically_damped, color = RED, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, 12]))
            y_tracker.set_value(-1+y_critically_damped(12))
            spring.add_updater(spring_direct_updater)
            self.play(FadeOut(sol_critically_damped), Unwrite(text_case3), run_time = 1.5)
            self.play(y_tracker.animate.set_value(x0-1), rate_func = smooth, run_time = 2)
            self.play(Transform(medium_critically_damped, medium_overdamped), run_time = 3)
            self.wait(0.5)
            self.play(Write(text_case4), run_time = 1.5)
            self.wait(1.5)
            self.remove_updater(spring_direct_updater)
            y_tracker.set_value(0)
            spring.add_updater(spring_overdamped_updater)
            self.add(sol_overdamped)
            self.play(y_tracker.animate.set_value(12), rate_func = linear, run_time = 6)
            self.wait(3)


# PART F
        def PartF():
            self.add(spring, spring_anchor, spring_base_length, spring_mass, text_head_deq2, npla_group, medium_overdamped, sol_overdamped, text_case4)
            spring.become(Spring(start = anchor_pos, end = np.array([dot_pos[0], y_overdamped(12) - 1, 0]), k = 0.5, tip_buff = 0.5, nodes = 20))
            spring_mass.become(Dot(np.array([dot_pos[0], y_overdamped(12) - 1, 0]), color = RED, radius = 0.1))
            self.play(Unwrite(text_case4), Unwrite(text_head_deq2), FadeOut(sol_overdamped), FadeOut(spring), FadeOut(spring_anchor), FadeOut(spring_base_length), FadeOut(spring_mass), FadeOut(medium_overdamped), run_time = 1.5)
            self.wait(1)
            self.play(Write(text_head_ho), Transform(npla_group, npla2_group), run_time = 1.5)
            self.wait(5)
            self.add(sol_undamped, sol_underdamped, sol_critically_damped, sol_overdamped)
            sol_undamped.add_updater(sol_updater)
            y_tracker.set_value(0)
            self.play(y_tracker.animate.set_value(12), rate_func = linear, run_time = 6)
            self.wait(10)


        y_tracker = ValueTracker(0)
        PartF()

Output saved by overwring previous file at animations/spring/harmonic_oscillator_GER_F.mp4.
